# Object oriented programming

Rust's design is highly influenced by functional languages, and features we discuss here (traits and polymorphism) are more close to their equivalents in Haskell and ML languages than to C++ and Java. But since Rust supports mutable variables, it can provide an object oriented like experience.

In Rust we have C structs:

In [2]:
struct Human {
    name: &'static str,
    age: u32,
}

let hamid = Human { name: "hamid", age: 22 };
hamid.age

22

We can define functions in the namespace of types, with `impl` keyword:

In [3]:
impl Human {
    // Rust doesn't have constructor, but we can use functions for it:
    fn new(name: &'static str, age: u32) -> Human {
        Human { name, age } // shorthand for `name: name, age: age`
    }

    fn new_baby(name: &'static str) -> Human {
        Human { name, age: 0 }
    }
}

let danial = Human::new("danial", 24);
danial.age

24

Namespaced functions can be completely unrelated to the type:

In [4]:
impl Human {
    fn a_completely_unrelated_function(x: i32) -> i32 {
        x + 5
    }
}

Human::a_completely_unrelated_function(10)

15

First argument of such function can be have a special keyword, `self`, which make the function a method:

In [5]:
impl Human {
    fn say_hello(self: &Human) {
        println!("{} says hello!", self.name);
    }
}

(&hamid).say_hello();
// compiler will add & automatically
hamid.say_hello();
// normal call is possible
Human::say_hello(&danial);

hamid says hello!
hamid says hello!
danial says hello!


Unlike normal functions, method type is restricted. It can be `Human`, `&Human`, `&mut Human`, `Box<Human>` and similar:

In [6]:
impl Human {
    fn unrelated(self: &i32) {
        println!("{}", self);
    }
}

Error: invalid `self` parameter type: &i32

`&mut self` is a shorthand for `self: &mut Human`:

In [7]:
impl Human {
    fn pass_a_year(&mut self) {
        self.age += 1;
    }
}

let mut baby = Human::new_baby("ali");
baby.pass_a_year();
baby.pass_a_year();
baby.pass_a_year();
baby.age

3

## Privacy and encapsulation

Rust uses module level privacy. By default, everything is only visible to the current module:

In [3]:
mod a {
    struct Foo {
        field: i32,
    }
}

let x: a::Foo;

Error: struct `Foo` is private

Error: the struct `Foo` is defined here

We can mark the struct as public, but it doesn't make fields public:

In [6]:
mod a {
    pub struct Foo {
        field: i32,
    }
}

let x: a::Foo = a::Foo { field: 5 };

Error: field `field` of struct `Foo` is private

If we make the field public as well, it will compile:

In [8]:
mod a {
    pub struct Foo {
        pub field: i32,
    }
}

let x: a::Foo = a::Foo { field: 5 };
x.field

5

So we can have a vector implementation which encapsulates the capacity and length so foreign user can't corrupt them: 

In [13]:
mod my_vec {
    pub struct MyVec {
        capacity: usize,
        length: usize, // fields are private
    }

    impl MyVec {
        // methods are public
        pub fn push(&mut self, _item: i32) {
            // we ignore item, because implementing an actual vector is hard
            if self.length == self.capacity {
                self.capacity *= 2;
                println!("capacity raised to {}", self.capacity);
            }
            self.length += 1;
            println!("length raised to {}", self.length);
        }

        pub fn len(&self) -> usize {
            self.length // this is like a getter. By making a getter public instead of
            // whole field, we can enforce invariants on it
        }

        pub fn new() -> MyVec {
            MyVec { length: 0, capacity: 1 }
        }
    }
}

use my_vec::MyVec;

let mut x = MyVec::new();
x.push(1);
x.push(2);
x.push(3);
x.push(4);
x.push(5);
x.len()

length raised to 1
capacity raised to 2
length raised to 2
capacity raised to 4
length raised to 3
length raised to 4
capacity raised to 8
length raised to 5


5

Changing length manually isn't possible:

In [14]:
x.length = 12;

Error: field `length` of struct `MyVec` is private

The real vector from the standard library is using the exact same tactic:

In [15]:
let mut x: Vec<i32> = vec![1, 2, 3];
x.push(4);
x.len = 12;

Error: field `len` of struct `Vec` is private

## Polymorphism

Polymorphism is another fruit of object oriented programming. It refers to the ability of calling function which needs a variable of `Base` class, with an instance of `Child1` and `Child2` class when both extends `Base` class. In Rust, we get polymorphism using `trait`. `trait` is similar to abstract class in C++ or interface in Java. Here is a simple trait:

In [2]:
trait Animal {
    // A pure virtual function, need to be implemented by implementor
    fn say_hello(&self);
    
    // A function with default, but implementor can override it as well
    fn say_hello_n_times(&self, n: u32) {
        for _ in 0..n {
            self.say_hello();
        }
    }
}

We can implement traits for types:

In [9]:
struct Dog(&'static str);

impl Animal for Dog {
    fn say_hello(&self) {
        println!("Hi! I'm dog {}, woof woof!", self.0);
    }
}

let dog = Dog("joe");
dog.say_hello_n_times(3);

Hi! I'm dog joe, woof woof!
Hi! I'm dog joe, woof woof!
Hi! I'm dog joe, woof woof!


In [10]:
struct Cat(&'static str);

impl Animal for Cat {
    fn say_hello(&self) {
        println!("Hi! I'm cat {}, meow meow!", self.0);
    }

    fn say_hello_n_times(&self, n: u32) {
        print!("Hi! I'm cat {},", self.0);
        for _ in 0..n {
            print!(" meow");
        }
        println!("!");
    }
}

let cat = Cat("donald");
cat.say_hello_n_times(5);

Hi! I'm cat donald, meow meow meow meow meow!


Now we want a vector, contain both cats and dogs:

In [26]:
let v = vec![Cat("donald"), Dog("joe"), Cat("george")];

Error: mismatched types

## Trait objects

By adding a `dyn` before a trait name, we can create a unsized type:

In [6]:
use std::mem::size_of;

In [7]:
size_of::<dyn Animal>();

Error: the size for values of type `dyn Animal` cannot be known at compilation time

So a pointer to it is a fat pointer:

In [8]:
size_of::<&dyn Animal>()

16

At binary level, a pointer to a `dyn Trait` is a pointer to the bytes of underlying type, and a pointer to a vtable. A vtable itself is a table, contains size of the type, and function pointers to the implementations of the trait functionality. When we call a trait function on a trait object, it will lookup on its vtable and call the corresponding function pointer.

So we can use trait object to make a function accept every type which implements the trait:

In [11]:
fn say_hello_2x(obj: &dyn Animal) {
    obj.say_hello();
    obj.say_hello();
}

say_hello_2x(&cat);
say_hello_2x(&dog);

Hi! I'm cat donald, meow meow!
Hi! I'm cat donald, meow meow!
Hi! I'm dog joe, woof woof!
Hi! I'm dog joe, woof woof!


Since `dyn Trait` is unsized, we can't store it directly in a vector:

In [29]:
let v: Vec<dyn Animal>;

Error: the size for values of type `dyn Animal` cannot be known at compilation time

So we need to store a pointer to them, like a reference:

In [13]:
let animals: Vec<&dyn Animal> = vec![
    &Cat("donald"),
    &Dog("joe"),
    &Cat("george"),
];

for animal in animals {
    animal.say_hello_n_times(4);
}

Hi! I'm cat donald, meow meow meow meow!
Hi! I'm dog joe, woof woof!
Hi! I'm dog joe, woof woof!
Hi! I'm dog joe, woof woof!
Hi! I'm dog joe, woof woof!
Hi! I'm cat george, meow meow meow meow!


()

But simple references are limited. For example, we can't return them from a function:

In [16]:
fn return_some_animals() -> Vec<&'static dyn Animal> {
    let cat1 = Cat("donald");
    let dog1 = Dog("joe");
    vec![
        &cat1,
        &dog1,
        &Cat("george"),
    ]
}

Error: cannot return value referencing local variable `cat1`

Error: cannot return value referencing local variable `dog1`

So for maximum flexibility, we can use a box, which heap allocates the space for the object, and owns the data:

In [17]:
fn return_some_animals() -> Vec<Box<dyn Animal>> {
    let cat1 = Cat("donald");
    let dog1 = Dog("joe");
    vec![
        Box::new(cat1),
        Box::new(dog1),
        Box::new(Cat("george")),
    ]
}

for animal in return_some_animals() {
    animal.say_hello_n_times(4);
}

Hi! I'm cat donald, meow meow meow meow!
Hi! I'm dog joe, woof woof!
Hi! I'm dog joe, woof woof!
Hi! I'm dog joe, woof woof!
Hi! I'm dog joe, woof woof!
Hi! I'm cat george, meow meow meow meow!


()

In Rust we usually prefer to not use trait object if possible, because performance:
* They are usually (but not always) tied with `Box`, so they need heap allocation
* Compiler doesn't have information about the specific function that will run in runtime, so it can't optimize well.

`dyn Trait` is the tool for dynamic dispatch, and Rust has tools for static dispatch as well.

## Generics

Definitions (functions, structs, traits, implementations, ...) can be generic over lifetimes (which you have seen in previous chapter), types, and constants. Let's see a generic function over types:

In [33]:
fn third<T>(x: &[T]) -> &T {
    &x[2]
}

println!("{}", third(&[1, 2, 3]));
println!("{}", third(&['a', 'b', 'c']));
println!("{}", third(&["hi", "hello", "bye"]));

3
c
bye


Unlike lifetimes which can be erased for execution, types are needed in runtime. Compiler uses monomorphization and make a copy of the function for each used type (very similar to templates in C++) but in semantics, generics are very different from templates.

Generics are type checked at definition, not at monomorphization:

In [34]:
fn third<T>(x: &[T]) -> &i32 {
    &x[2]
}

Error: mismatched types

So we can't do anything useful with them:

In [35]:
fn say_hello_2x<T>(x: T) {
    x.say_hello();
    x.say_hello();
}

Error: no method named `say_hello` found for type parameter `T` in the current scope

Error: no method named `say_hello` found for type parameter `T` in the current scope

Unless we apply the compiler's suggestion and restrict the type:

In [40]:
fn say_hello_2x<T: Animal>(x: T) {
    x.say_hello();
    x.say_hello();
}

say_hello_2x(Cat("donald"));
say_hello_2x(Dog("joe"));

Hi! I'm cat donald, meow meow!
Hi! I'm cat donald, meow meow!
Hi! I'm dog joe, woof woof!
Hi! I'm dog joe, woof woof!


Calling the function with bad argument will fail, before monomorphization. It is because compiler knows the restriction from the function signature:

In [42]:
fn say_hello_2x<T: Animal>(x: T) {
    x.say_hello();
    x.say_hello();
}

say_hello_2x("not an animal");

Error: the trait bound `&str: Animal` is not satisfied

So there is no cryptic monomorphization error in Rust. The generic system of Rust is considered superior to C++ templates, because of better errors, better IDE support, strong type checking, and the property "changing a function implementation should not break consumers" which is very important for semver, and is not guaranteed in templates.

C++20 added concepts and constraints which can bring some of the benefits of Rust generics to the C++ template system.

## Method resolution

What will happen if there are two traits with a single method which a single type implements both?

In [2]:
struct Type;
trait Foo1 {
    fn foo(&self) -> &'static str {
        "Trait foo1 called"
    }
}

trait Foo2 {
    fn foo(&self) -> &'static str {
        "Trait foo2 called"
    }
}

impl Foo1 for Type {}
impl Foo2 for Type {}

In [3]:
let x = Type;
x.foo()

Error: candidate #1 is defined in an impl of the trait `Foo1` for the type `Type`

It is ambiguous and it isn't allowed to call such method directly in Rust. We can call them using `Trait::method` syntax:

In [4]:
let x = Type;
Foo1::foo(&x)

"Trait foo1 called"

Or with `<Type as Trait>::method` syntax:

In [6]:
let x = Type;
<Type as Foo2>::foo(&x)

"Trait foo2 called"

But methods with same name are not a big problem in practice, since we should import each trait if we want to call its methods:

In [10]:
mod module {
    pub struct Type2;
    pub trait Bar {
        fn bar(&self) -> &'static str {
            "Trait bar called"
        }
    }
    impl Bar for Type2 {}
}

In [11]:
use module::Type2;
let x = Type2;
x.bar()

Error: no method named `bar` found for struct `Type2` in the current scope

We need to explicitly import the trait `Bar` in order to call its methods:

In [12]:
use module::{Type2, Bar};
let x = Type2;
x.bar()

"Trait bar called"

So if we import one of the `Foo1` or `Foo2` in the above example, we can use the method syntax.

Methods on `dyn Trait` always resolve to methods on `Trait`:

In [15]:
let x: Box<dyn Foo2> = Box::new(Type);
// The object inside x implements `Foo1` as well, but compiler doesn't care
x.foo()

"Trait foo2 called"

And the same with generics:

In [17]:
fn call_foo_on_foo1<T: Foo1>(x: T) -> &'static str {
    x.foo() // This is always foo of `Foo1`, even if `T` is instantiated with types with other methods.
}
let x = Type;
call_foo_on_foo1(x)

"Trait foo1 called"

As sail above, generics in Rust are not macro-ish templates like C++. Type checking and name resolution is done on the generic item and monomorphization is just a code generation strategy.

## Associated types

In addition to virtual methods, traits can have associated types, which implementations define them:

In [18]:
trait Holder {
    type HeldItem;
    fn get_item(&self) -> &Self::HeldItem;
    fn set_item(&mut self, new_item: Self::HeldItem);
}

In [19]:
struct I32Holder {
    value: i32,
}

impl Holder for I32Holder {
    type HeldItem = i32;
    fn get_item(&self) -> &i32 {
        &self.value
    }
    fn set_item(&mut self, new_item: i32) {
        self.value = new_item;
    }
}

In [20]:
let x = I32Holder { value: 5 };
x.get_item()

5

In [21]:
struct CatCage {
    member: Cat,
}

impl Holder for CatCage {
    type HeldItem = Cat;
    fn get_item(&self) -> &Cat {
        &self.member
    }
    fn set_item(&mut self, new_item: Cat) {
        self.member = new_item;
    }
}

In [10]:
let cage = CatCage { member: Cat("cathy") };
cage.get_item().say_hello();

Hi! I'm cat cathy, meow meow!


We can use `Trait<AssocType=Ty>` syntax to define what associated type we expect:

In [29]:
fn set_held_value_to_5(holder: &mut dyn Holder<HeldItem = i32>) {
    holder.set_item(5);
}

let mut x = I32Holder { value: 2 };
println!("{}", x.get_item());
set_held_value_to_5(&mut x);
println!("{}", x.get_item());

2
5


Or we can use them with generics:

In [22]:
fn say_hello_2x_by_held_item<T>(holder: T)
where
    T: Holder,
    T::HeldItem: Animal,
{
    let item: &T::HeldItem = holder.get_item();
    item.say_hello();
    item.say_hello();
}

say_hello_2x_by_held_item(CatCage { member: Cat("cathy") });

Hi! I'm cat cathy, meow meow!
Hi! I'm cat cathy, meow meow!


As always, generics are fully type checked and you won't get a monomorphization error (or worse, a successful call which doesn't satisfy restrictions):

In [23]:
say_hello_2x_by_held_item(I32Holder { value: 5 });

Error: the trait bound `i32: Animal` is not satisfied

## Orphan rule

We can implement other traits for our types. Here, `From` is a trait in the standard library for converting a type to another in a lossless way (like `i32` to `i64`) and we can implement it for our cat:

In [30]:
use std::convert::From;

impl From<&'static str> for Cat {
    fn from(value: &'static str) -> Cat {
        Cat(value)
    }
}

let y: Cat = "goofy".into(); // provided by `From` trait.
y.say_hello();

Hi! I'm cat goofy, meow meow!


Which is also possible in C++. In addition to that, We can implement our traits for other types:

In [3]:
impl Animal for i64 {
    fn say_hello(&self) {
        println!("Hello, I'm an i64 {self}. I'm not really an animal.");
    }
}

1234567890123.say_hello();

Hello, I'm an i64 1234567890123. I'm not really an animal.


Which is not possible in C++. But it is not completely unlimited. You can't implement traits defined in other crates for types defined in other crates:

In [5]:
impl From<String> for Vec<i32> {
    fn from(value: String) -> Vec<i32> {
        todo!()
    }
}

Error: only traits defined in the current crate can be implemented for types defined outside of the crate

This rule prevents breakage while adding a dependency, which might happen in C and C++ due name conflicts in includes and template errors.

## Super traits

Each trait can have super traits. For example, here `Animal` is super trait of `Mammal` (and `Mammal` is sub trait of `Animal`):

In [14]:
trait Mammal: Animal {
    fn feed_children(&self);
}

We can implement `Mammal` for every type that implements `Animal`:

In [ ]:
impl Mammal for Cat {
    fn feed_children(&self) {
        println!("Cat {} is feeding children", self.0);
    }
}

In [19]:
Cat("cathy").feed_children();

Cat cathy is feeding children


But implementing it for types that don't implement `Animal` will result in a type error:

In [20]:
impl Mammal for i32 {
    fn feed_children(&self) {
        
    }
}

Error: the trait bound `i32: Animal` is not satisfied

We can run methods of super trait on the type constrained with sub trait, and trait objects:

In [25]:
fn say_hello_10x_and_feed(x: &dyn Mammal) {
    x.say_hello_n_times(10);
    x.feed_children();
}

say_hello_10x_and_feed(&Cat("cathy"));

Hi! I'm cat cathy, meow meow meow meow meow meow meow meow meow meow!
Cat cathy is feeding children


But super trait is not inheritance. It's just a precondition for types that implements a trait. To better see it, we can use this more verbose syntax:

In [15]:
trait Mammal2
where
    Self: Animal
{
    fn feed_children(&self);
}

Preconditions in `where` section can be arbitary and complex:

In [17]:
trait SomeTrait
where
    Self: Animal + Holder, // Both Animal and Holder are super traits
    <Self as Holder>::HoldedItem: Animal,
{
    fn some_method(&self);
}

## Special language traits

There are several traits that language uses by special syntax and semantic. For example, operator overloading is done using traits:

In [47]:
#[derive(Debug)]
enum Color {
    Red,
    Green,
    Blue,
    Cyan,
    Magenta,
    Yellow,
    White,
}

use Color::*;

impl std::ops::Add<Color> for Color {
    type Output = Color; // traits can have associated types and associated constants, in addition to functions
    fn add(self, other: Color) -> Color {
        match (self, other) {
            (Red, Green) | (Green, Red) => Yellow,
            (Red, Blue) | (Blue, Red) => Magenta,
            (Green, Blue) | (Blue, Green) => Cyan,
            _ => unimplemented!(),
        }
    }
}

Red + Blue

Magenta

Similar traits exist for `*`, `/`, `-`, `&`, `|`, `!`, `==`, `<` but not for `&&`, `||`, and notably, `=`. Short circuit operators are boolean specific, and assignment operator is not overloadable to make things simple and predictable. It always do the move by bitwise memcpy.

Drop trait allows type to run code on destruct. We have seen it in the previous chapter:

In [48]:
struct DebugDrop(&'static str);
impl Drop for DebugDrop {
    fn drop(&mut self) {
        println!("{} has been dropped", self.0);
    }
}
{
    let x = DebugDrop("test");
    // it should print here
}
println!("finished");

test has been dropped
finished


Since calling drop method twice can lead to double free or use after free problems, only compiler is able to call drop method: 

In [49]:
let mut v = vec![1, 2, 3];
v.drop();

Error: explicit use of destructor method

Compiler suggests using `drop` function, which is nothing but an empty function that takes the argument:

In [51]:
fn my_drop<T>(_arg: T) {
}

{
    let x = DebugDrop("test1");
    let y = DebugDrop("test2");
    my_drop(y); // exactly equal to drop(y);
    println!("Before the end of the scope");
};

test2 has been dropped
Before the end of the scope
test1 has been dropped


`Debug` and `Display` are another special types, used in macros that use formatted arguments, like `println!`: 

In [64]:
enum MathOperator {
    Plus,
    Minus,
    Product,
    Div,
}

use MathOperator::*;
use std::fmt::{Display, Debug, Formatter, Error};

impl Display for MathOperator {
    fn fmt(&self, f: &mut Formatter<'_>) -> Result<(), Error> {
        let result = match self {
            Plus => "+",
            Minus => "-",
            Product => "*",
            Div => "/",
        };
        write!(f, "{result}")
    }
}

impl Debug for MathOperator {
    fn fmt(&self, f: &mut Formatter<'_>) -> Result<(), Error> {
        let result = match self {
            Plus => "plus",
            Minus => "minus",
            Product => "product",
            Div => "div",
        };
        write!(f, "{result}")
    }
}

println!("{} and {:?} are two math operator", Plus, Minus);

+ and minus are two math operator


`IntoIterator` is the trait that compiler uses for desugaring for loops. We will look at iterators in depth in the next chapter, but to show that compiler is actually using this trait for `for` loop, we can use this:

In [65]:
fn for_and_print<T>(into_iter: T) {
    for item in into_iter {
        println!("{}", item);
    }
}

Error: `T` is not an iterator

Compiler here is suggesting to restrict `T` to iterators, which generally make sense, but not for showing that `for` is using `IntoIterator`. Let's not listen to the compiler this time:

In [67]:
fn for_and_print<T: IntoIterator>(into_iter: T) {
    for item in into_iter {
        println!("{}", item);
    }
}

Error: `<T as IntoIterator>::Item` doesn't implement `std::fmt::Display`

Rust's compiler errors are great, aren't? Compiler not only understands that iterator's items are not always displayable, but tells us the syntax to fix that. 

In [69]:
fn for_and_print<T>(into_iter: T)
where
    T: IntoIterator,
    <T as IntoIterator>::Item: Display,
{
    for item in into_iter {
        print!("{} ", item);
    }
    println!();
}

for_and_print(5..10);
for_and_print(vec!['a', 'b', 'e']);

5 6 7 8 9 
a b e 


`Clone` trait plays the rule of copy constructors in C++. Though it is not a special trait in the language. You need to explicitly call `.clone()` everywhere you need cloning, like any other trait.

In [70]:
let v1 = vec![1, 2, 3];
let v2 = v1; // move
let v3 = v2.clone(); // creates a new vector, equal to the previous one
let v4 = Clone::clone(&v2); // like above, but with namespace syntax
let v5 = Vec::clone(&v2); // Trait functions are available at type's namespace as well
println!("{v2:?} {v3:?} {v4:?} {v5:?}");

[1, 2, 3] [1, 2, 3] [1, 2, 3] [1, 2, 3]


`Copy` trait tells the compiler that the source is not invalid after the move. It doesn't have any method. Traits without methods are called marker traits. Since declaring wrong types (like a type that has a vector in a field) voids the memory safety of Rust, the only way to implement `Copy` for your type in safe Rust is using `derive`.

## Derive

For some trait, there is a general way to make a reasonable implementation. For example, we can `.clone` everything for implementing `Clone`, checking if every field is equal to the corresponding field for `Eq`, printing struct name and each field for debug, ... . There are some macros that can do this for us, automatically:

In [73]:
#[derive(Debug, Clone, PartialEq, Eq, Default)]
struct SomeStruct {
    field1: Vec<i32>,
    field2: &'static str,
}

let mut x = SomeStruct::default(); // provided by `Default` derived implementation
println!("default is {:?}", x); // `{:?}` is available for `Debug` implementation
let y = x.clone();
x.field2 = "hello";
println!("x after change is {:?}", x);
let z = x.clone();
println!("x == y is {}", x == y); // `==` is available for `PartialEq` implementation
println!("x == z is {}", x == z);

default is SomeStruct { field1: [], field2: "" }
x after change is SomeStruct { field1: [], field2: "hello" }
x == y is false
x == z is true


You can implement `Copy` trait for your type with `derive`:

In [76]:
#[derive(Clone, Copy)] // Clone is necessary, because it is a super trait of copy
struct SomeCopyableStruct {
    field1: i32,
    field2: &'static str,
}

let x = SomeCopyableStruct { field1: 5, field2: "hi" };
let y = x;
// x is still available
x.field1

5

You will get a compiler error if you try to derive `Copy` if fields are not `Copy`. It is not specific to `Copy` and you will get errors for other traits like `Clone`, `Default`, `Debug`, ... too.

In [77]:
#[derive(Clone, Copy)]
struct SomeCopyableStruct {
    field1: Vec<i32>,
    field2: &'static str,
}

Error: the trait `Copy` may not be implemented for this type

Derive is not specific to the standard library traits. For example, serde is a third party crate for serializing to formats like `json`, `yaml`, `pickle` (python's serialization format) `cbor` (a binary format) and many other formats. Serde provides derive macros for it's traits, `Serialize` and `Deserialize`:

In [80]:
:dep serde = { version = "1.0", features = ["derive"] }
:dep serde_json = "1.0"
:dep serde_yaml = "0.9"

In [83]:
use serde::{Serialize, Deserialize};

#[derive(Serialize, Deserialize)]
struct Complex {
    field1: Vec<i32>,
    field2: NestedStruct,
}

#[derive(Serialize, Deserialize)]
struct NestedStruct {
    field1: i32,
    field2: f64,
}

let x = Complex {
    field1: vec![1, 2, 3],
    field2: NestedStruct {
        field1: 5,
        field2: 12.3,
    },
};

println!("{}", serde_json::to_string(&x).unwrap());

{"field1":[1,2,3],"field2":{"field1":5,"field2":12.3}}


In [84]:
println!("{}", serde_yaml::to_string(&x).unwrap());

field1:
- 1
- 2
- 3
field2:
  field1: 5
  field2: 12.3



Deriving traits is a very useful feature in Rust, inherited from its functional parents like Haskell. C++ doesn't have this feature (yet). The alternative in the C++ world is code generation, which doesn't worth the effort for many cases.

## Auto traits

There are some traits, which compiler will automatically implements them for any type, to not make Rust code unnecessarily verbose.

`Send` trait is a marker trait, which is automatically implemented for each type when all fields are `Send`. Implementing `Send` means this type is safe to move between threads. For example, `Rc` is not `Send`, because if we move an instance of it between threads, one thread can drop it and decrease it's reference count with another thread simultaneously, causing a data race, which is impossible.

In [91]:
use std::{thread, rc::Rc};

In [92]:
let x = Rc::new(5);
thread::spawn(move || {
    println!("{x}");
});

Error: `Rc<i32>` cannot be sent between threads safely

So if we need to send a generic value between threads, we need to bound it with send:

In [97]:
fn send_anything<T>(t: T) {
    thread::spawn(move || {
        let _x = t; // move t to thread and ignore it.
    });
}

Error: `T` cannot be sent between threads safely

`Sync` is a similar auto trait, which states that a type can be shared by reference across threads. `Rc` is not sync as well, `RefCell` is not `Sync`, but is `Send`.

`Sized` is another auto trait, which is implemented for every type which is not unsized (`[u8]`, `str`, `dyn Trait`, ...) `Sized` is special in another way as well: It is assumed for every generic parameter by default. For opt out of this, we can use `?Sized` bound, which means either sized or unsized:

In [98]:
fn foo<T: ?Sized>(x: T) {
    
}

Error: the size for values of type `T` cannot be known at compilation time

## Lifetimes as traits

Do you remember `'b: 'a` syntax which means `'b >= 'a`? You can view it as `'a` is a super lifetime of `'b` and anything which lives longer than `'b` (and so it is valid for every point in `'b`) should also live longer than `'a`. So we can consider `'x` as a trait which is implemented for every type which is guaranteed to be valid for every point in `'x`. And there are other syntaxes related to lifetimes which follow this mindset. For example, `T: 'a` can be used to show if every instance of `T` can be valid during `'a`. Let's check it:

In [34]:
macro_rules! assert_impl {
    ($($x:tt)*) => {
        struct F where $($x)*; // Reference with lifetime 'a is not 'static
        let _x = F;
    }
}

In [37]:
assert_impl!(i32: 'static); // i32 can be valid during whole program

In [38]:
assert_impl!(Vec<Vec<Option<i32>>>: 'static); // Vec of static types can be valid during whole program

In [35]:
assert_impl!(for<'a> &'a i32: 'static); // Reference with lifetime 'a is not 'static

Error: higher-ranked lifetime error

In [39]:
assert_impl!(for<'a> &'a i32: 'a); // Reference with lifetime 'a is 'a

In [43]:
assert_impl!(for<'a> &'static &'a i32: 'static);

Error: higher-ranked lifetime error

In [46]:
assert_impl!(i32: Clone + Copy + Animal + 'static); // You can check multiple bound at once, mix trait and lifetimes

Error: the trait bound `i32: Animal` is not satisfied

But what is the usage of this? One use case is declaring lifetimes for `dyn Trait` objects:

In [53]:
use std::fmt::Debug;
use std::sync::Mutex;

In [57]:
static LongTimeStorage: Mutex<Option<Box<dyn Debug + Send>>> = Mutex::new(None);

fn put_box_into_an_static_item<'a>(value: Box<dyn Debug + Send + 'a>) {
    *LongTimeStorage.lock().unwrap() = Some(value);
}

Error: unused macro definition: `assert_impl`

Error: lifetime may not live long enough

In [58]:
static LongTimeStorage: Mutex<Option<Box<dyn Debug + Send>>> = Mutex::new(None);

fn put_box_into_an_static_item(value: Box<dyn Debug + Send + 'static>) {
    *LongTimeStorage.lock().unwrap() = Some(value);
}

By `'static` condition that appears in signature of `put_box_into_an_static_item` compiler can prevent usages that have memory problems:

In [61]:
{
    let x = 2;
    put_box_into_an_static_item(Box::new(&x)); // &'lifetime_of_x i32: 'static is false
    // x is dropped here
}
// will dereference an invalid reference to x!!!
println!("{:?}", LongTimeStorage);

Error: unused macro definition: `assert_impl`

Error: `x` does not live long enough

But safe usage is allowed:

In [62]:
{
    put_box_into_an_static_item(Box::new(2)); // we have i32: 'static
}
println!("{:?}", LongTimeStorage);

Mutex { data: Some(2), poisoned: false, .. }


Another example of using `Type: 'lifetime` bound is leak function. Consider this leak function which move a thing to heap and leak it:

In [69]:
fn leak<T>(x: T) -> &'static T {
    Box::leak(Box::new(x))
}

We can't use this function for leaking references:

In [70]:
{
    let x = 2;
    let r: &&i32 = leak(&x);
    **r
}

Error: unused macro definition: `assert_impl`

Error: `x` does not live long enough

The problem here is that a reference to a temporary can't live for whole program, because it's pointee will be invalidated at some time. Compiler implicitly added a `T: 'static` to our signature and with that it reject out code above. But we can relax our leak function signature to allow it to accept references and other non static types:

In [72]:
fn leak<'a, T>(x: T) -> &'a T
where
    T: 'a
{
    Box::leak(Box::new(x))
}

Here the lifetime is not necessarily static, but it is some maximal lifetime that input type can allow. Now the above code would work:

In [73]:
{
    let x = 2;
    let r: &&i32 = leak(&x);
    **r
}

2

## Impl trait

So we can write a runtime polymorphic function with `dyn Trait`:

In [4]:
use std::fmt::Debug;

fn foo(arg: &dyn Debug) {
    println!("{arg:?} - {arg:?} - {arg:?}");
}

foo(&2);
foo(&"hello");

2 - 2 - 2
"hello" - "hello" - "hello"


And compile time version with generics:

In [6]:
fn foo<T: Debug>(arg: T) {
    println!("{arg:?} - {arg:?} - {arg:?}");
}

foo(2);
foo("hello");

2 - 2 - 2
"hello" - "hello" - "hello"


The generic version is more performant, but the `dyn Trait` version is more beautiful. The `T` in the generic version is completely useless and `arg: T` says nothing about what arg exactly is. So to prevent people from using `dyn Trait` as a code style choice, Rust has `impl Trait` which is exactly equivalent to generics:

In [7]:
use std::fmt::Debug;

fn foo(arg: impl Debug) {
    println!("{arg:?} - {arg:?} - {arg:?}");
}

foo(2);
foo("hello");

2 - 2 - 2
"hello" - "hello" - "hello"


`impl Trait`, unlike `dyn Trait`, is not a real type. For example, we can't use it in type name:

In [8]:
std::any::type_name::<impl Debug>()

Error: `impl Trait` only allowed in function and inherent method return types, not in path

But this is fine:

In [9]:
std::any::type_name::<dyn Debug>()

"dyn core::fmt::Debug"

Generics are strictly more powerful than `impl Trait` in argument position. For example, this code:

In [10]:
fn print_and_push_to_vec<T: Debug>(vec: &mut Vec<T>, input: T) {
    println!("{input:?}");
    vec.push(input);
}

let mut v = vec![1, 2, 3];
print_and_push_to_vec(&mut v, 5);
v

5


[1, 2, 3, 5]

Can not be expressed using `impl Trait`:

In [13]:
fn print_and_push_to_vec(vec: &mut Vec<impl Debug>, input: impl Debug) {
    println!("{input:?}");
    vec.push(input);
}

Error: mismatched types

Because the above is equivalent to this code:

In [14]:
fn print_and_push_to_vec<A: Debug, B: Debug>(vec: &mut Vec<A>, input: B) {
    println!("{input:?}");
    vec.push(input);
}

Error: mismatched types

Which is clearly wrong.

But there is another usage of `impl Trait` which can't be expressed by generics. Normally, you can't make types in your public interface private:

In [15]:
mod privacy_boundary {
    struct Foo;

    pub fn some_f() -> Foo {
        Foo
    }
}

Error: private type `Foo` in public interface

But you may want to guarantee that a function's return type only supports a trait, and nothing more. In this case, you can use return position `impl Trait`:

In [19]:
mod privacy_boundary {
    #[derive(Debug)]
    struct Foo;

    pub fn some_f() -> impl std::fmt::Debug {
        Foo
    }
}

{
    let x = privacy_boundary::some_f();
    println!("{x:?}");
};

Foo


The generic version has a very different meaning. `impl Trait` means this function returns some opaque type which implements `Trait`, but generic version means this function is able to generate an instance for each type that implements `Trait`, which our function certainly can't:

In [20]:
#[derive(Debug)]
struct Foo;

pub fn some_f<T: Debug>() -> T {
    Foo
}

Error: mismatched types

The generic version has it's own usage, for example:

In [21]:
pub fn default_tuple<T: Default>() -> (T, T) {
    (T::default(), T::default())
}

default_tuple::<[String; 3]>()

(["", "", ""], ["", "", ""])

But it has very different meaning from `impl Trait` in return position.